In [ ]:
import requests
import json
import re
import pandas as pd

# 網頁 URL
fUrl = "https://famihealth.family.com.tw/Calculator"

# 發送 GET 請求
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
response = requests.get(fUrl, headers=headers)

df = pd.DataFrame()
# 確保請求成功
if response.status_code == 200:
    # 使用正則表達式提取 categories 變數的內容
    match = re.search(r'var categories = (\[.*?\]);', response.text, re.S)
    if match:
        # 解析為 Python 字典
        categories_data = json.loads(match.group(1))
        
        # 存儲結果
        results = []
        for category in categories_data:
            for product in category.get("products", []):
                results.append({
                    "category": category.get("name"),
                    "title": product.get("name"),
                    "picture_url": product.get("imgurl"),
                    "Protein (g)": product.get("protein", 0),
                    "Carb (g)": product.get("carb", 0),
                    "Calories (kcal)": product.get("calo", 0),
                    "Fat (g)": product.get("fat", 0),
                    "Description": product.get("description", ""),
                })
        
        # 轉換為 DataFrame
        df = pd.DataFrame(results)


In [16]:
json_data = df.to_dict(orient='records')

# 儲存為 JSON 文件，並指定編碼為 UTF-8
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)